### Serena Williams
## Phase 3 - Implementing convolutional neural network
## Phase 4 - Use transfer learning to use pre-trained weights as initializations

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import keras


from keras.models import Sequential
from keras.layers import Dense, Conv2D, Flatten, MaxPooling2D

from sklearn.model_selection import train_test_split

from sklearn.metrics import confusion_matrix, f1_score, precision_score, recall_score
from keras.models import load_model
from keras.models import Model

Using TensorFlow backend.


In [2]:
X = pd.read_csv("X.csv", sep=' ', header=None, dtype=float)
X = X.values
print(X.shape)

(13233, 4096)


In [3]:
#reshaping the array
len(X)
X1 = np.zeros((13233,64,64))
for i in range(len(X)):
    X1[i] = X[i].reshape(64,64)

In [4]:
y = pd.read_csv("y_bush_vs_others.csv", header=None)
y_bush = y.values.ravel()
y = pd.read_csv("y_williams_vs_others.csv", header=None)
y_williams = y.values.ravel()

In [5]:
np.sum(y_williams)

52

In [6]:
print(X1.shape,y_bush.shape,y_williams.shape)

(13233, 64, 64) (13233,) (13233,)


In [7]:
seed1 = 3243
X_train, X_test, y_train, y_test = train_test_split(X1, y_williams, test_size = 1./3, random_state = seed1, shuffle = True)
X_train = X_train.reshape(8822,64,64,1)
X_test = X_test.reshape(4411,64,64,1)
print(X_train.shape,y_train.shape,X_test.shape)
# X_test = X_test.reshape(10000,28,28,1)

(8822, 64, 64, 1) (8822,) (4411, 64, 64, 1)


In [8]:
np.sum(y_test)

20

In [88]:
#Keras model 
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3), strides=(1, 1),
                 activation='relu',
                 input_shape=(64, 64, 1)))
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
# model.add(Conv2D(64, (5, 5), activation='relu'))
# model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
# model.add(Dense(128, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

### Loading the pre-trained model

In [23]:
model = load_model('tf_model.model')

In [98]:
model.input

model.summary(line_length=150)

model.layers.pop()
model.layers.pop()

model.summary(line_length=150)

inp = model.input
out =model.layers[-1].output

model2 = Model(inp, out)  # create a new model which doesn't have the last two layers in VGG16
model2.summary(line_length=150)

______________________________________________________________________________________________________________________________________________________
Layer (type)                                                       Output Shape                                                Param #                
conv2d_5 (Conv2D)                                                  (None, 64, 64, 8)                                           80                     
______________________________________________________________________________________________________________________________________________________
max_pooling2d_5 (MaxPooling2D)                                     (None, 32, 32, 8)                                           0                      
______________________________________________________________________________________________________________________________________________________
conv2d_6 (Conv2D)                                                  (None, 32, 32, 16)         

In [99]:
model.layers

In [25]:
for layer in model.layers[2:]:
    layer.trainable = False

In [26]:
model.layers[2:]

In [27]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
history = model.fit(X_train, y_train, epochs=20)
eval_metrics = model.evaluate(X_test, y_test)

Epoch 1/20
8822/8822 [==============================] - 2s 220us/step - loss: 0.0540 - acc: 0.9841
Epoch 2/20
8822/8822 [==============================] - 2s 182us/step - loss: 0.0241 - acc: 0.9965
Epoch 3/20
8822/8822 [==============================] - 2s 183us/step - loss: 0.0232 - acc: 0.9961
Epoch 4/20
8822/8822 [==============================] - 2s 181us/step - loss: 0.0239 - acc: 0.9963
Epoch 5/20
8822/8822 [==============================] - 2s 182us/step - loss: 0.0212 - acc: 0.9963 1s 
Epoch 6/20
8822/8822 [==============================] - 2s 180us/step - loss: 0.0220 - acc: 0.9966
Epoch 7/20
8822/8822 [==============================] - 2s 182us/step - loss: 0.0243 - acc: 0.9959
Epoch 8/20
8822/8822 [==============================] - 2s 181us/step - loss: 0.0245 - acc: 0.9960
Epoch 9/20
8822/8822 [==============================] - 2s 181us/step - loss: 0.0217 - acc: 0.9961
Epoch 10/20
8822/8822 [==============================] - 2s 180us/step - loss: 0.0233 - acc: 0.9961 1
Epo

In [13]:
print(eval_metrics)

[1.9433495320584113, 0.20788936755117227]


In [14]:
model.predict(X1[0].reshape(1,64,64,1))

array([[0.9850698]], dtype=float32)

In [15]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_5 (Conv2D)            (None, 64, 64, 8)         80        
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 32, 32, 8)         0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 32, 32, 16)        1168      
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 16, 16, 16)        0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 16, 16, 32)        4640      
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 8, 8, 32)          0         
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 8, 8, 64)          18496     
__________

In [16]:
pred = model.predict_classes(X_test.reshape(4411,64,64,1))
print(pred)

[[0]
 [0]
 [1]
 ...
 [0]
 [1]
 [1]]


In [28]:
pred = model.predict_classes(X_test.reshape(4411,64,64,1))
pred_train = model.predict_classes(X_train.reshape(8822,64,64,1))

In [29]:
print(np.sum(pred),np.sum(y_test))
print(np.sum(pred_train),np.sum(y_train))

1 20
0 32


In [19]:
#Test F1
val_f1 = f1_score(y_test, pred)
val_recall = recall_score(y_test, pred)
val_precision = precision_score(y_test, pred)
print(val_f1,val_recall,val_precision)

0.011318619128466326 1.0 0.005691519635742743


In [117]:
#Train f1
val_f1 = f1_score(y_train, pred_train)
val_recall = recall_score(y_train, pred_train)
val_precision = precision_score(y_train, pred_train)
print(val_f1,val_recall,val_precision)

0.896551724137931 0.8125 1.0


In [118]:
model.save("./phase 4/williams.model")

In [22]:
del model